### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [28]:
# Updated import for the installed AutoGen version
import os
from autogen import oai
from autogen.oai import OpenAIWrapper

# Create OpenAI client using the standard AutoGen approach
model_client = OpenAIWrapper(
    model="gpt-4.1-mini",  # Using gpt-4.1-mini as requested
    api_key=os.getenv("OPENAI_API_KEY")
)

In [18]:
# Ollama support - using standard AutoGen approach
# Note: Ollama integration may require additional setup
try:
    from autogen.oai import OpenAIWrapper
    # For Ollama, you would typically use a local endpoint
    ollamamodel_client = OpenAIWrapper(
        model="llama3.2",
        api_base="http://localhost:1234/v1",  # Ollama default endpoint
        api_key="ollama"  # Ollama doesn't require a real API key
    )
    print("✅ Ollama client created (requires Ollama to be running locally)")
except Exception as e:
    print(f"⚠️ Ollama setup required: {e}")
    ollamamodel_client = None

✅ Ollama client created (requires Ollama to be running locally)


### Second concept: The Message

In [19]:
# Updated import for the installed AutoGen version
from autogen.agentchat import ConversableAgent
import os

# In AutoGen, messages are typically handled through agent conversations
# Let's create a simple message structure
class SimpleMessage:
    def __init__(self, content, source="user"):
        self.content = content
        self.source = source
    
    def __repr__(self):
        return f"Message(content='{self.content}', source='{self.source}')"

message = SimpleMessage(content="I'd like to go to Rome", source="user")
message

Message(content='I'd like to go to Rome', source='user')

### Third concept: The Agent

In [20]:
# Updated import for the installed AutoGen version
from autogen.agentchat import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    llm_config={
        "model": "gpt-4.1-mini",
        "api_key": os.getenv("OPENAI_API_KEY")
    },
    system_message="You are a helpful assistant for an airline. You give short, humorous answers."
)

### Put it all together with on_messages

In [21]:
# Updated for the installed AutoGen version
# In AutoGen, we use the generate_reply method instead of on_messages
response = agent.generate_reply(
    messages=[{"role": "user", "content": message.content}],
    sender=agent
)
print("Agent response:", response)

Agent response: Ah, Rome! Where even the ruins have a better social life than me. Let’s book that trip!


### Let's make a local database of ticket prices

In [22]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [23]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [24]:
# Method to get price for a city
from typing import Union, Optional

def get_city_price(city_name: str) -> Optional[float]:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [25]:
get_city_price("Rome")

499.0

In [26]:
# Updated import for the installed AutoGen version
from autogen.agentchat import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    llm_config={
        "model": "gpt-4.1-mini",
        "api_key": os.getenv("OPENAI_API_KEY")
    },
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    function_map={"get_city_price": get_city_price}
)

In [29]:
# Updated for the installed AutoGen version
# In AutoGen, we use the generate_reply method instead of on_messages
response = smart_agent.generate_reply(
    messages=[{"role": "user", "content": message.content}],
    sender=smart_agent
)
print("Smart agent response:", response)

Smart agent response: Ah, Rome! Where even the ruins have style. A roundtrip ticket will cost you $850. Ready to get your Roman holiday on?
